In [1]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import re
from datetime import datetime
import pandas as pd
import os
from pathlib import Path

# Configurando as opções do Chrome para executar headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ativar o modo headless
service = ChromeService(ChromeDriverManager().install())




In [2]:


# driver = webdriver.Chrome(service=service)
driver = webdriver.Chrome(service=service, options=chrome_options)


url = "https://g1.globo.com/tudo-sobre/petrobras/"

driver.get(url)

time.sleep(5)

cookie = driver.find_element(
    By.XPATH, '//*[@id="cookie-banner-lgpd"]/div/div[2]/button'
)
cookie.click()


regex_data = r"/(\d{4}/\d{2}/\d{2})/"
formato = "%Y/%m/%d"


def busca_link(dados):
    link = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .get_attribute("href")
    )
    return link


def transforma_data(dados):
    match = re.search(regex_data, dados)
    if match:
        data_str = match.group(1)
        data = datetime.strptime(data_str, formato)
    else:
        data = None
    return data


def busca_link(dados):
    link = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .get_attribute("href")
    )
    return link


def busca_botao_veja_mais():
    veja_mais = driver.find_element(
        By.XPATH,
        "//*[@id='feed-placeholder']/div/div/div[3]/a[contains(text(), 'Veja mais')]",
    )
    return veja_mais


def busca_conteudo():
    conteudo = driver.find_elements(
        By.XPATH,
        '//*[contains(concat( " ", @class, " " ), concat( " ", "type-materia", " " ))]',
    )
    return conteudo


def obtem_titulo(dados):
    titulo = (
        dados.find_element(By.CLASS_NAME, "feed-post-body")
        .find_element(By.TAG_NAME, "a")
        .text
    )
    return titulo


limit = True
while limit:
    veja_mais = busca_botao_veja_mais()
    veja_mais.click()
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(5)
    conteudo = busca_conteudo()
    link = (
        busca_link(conteudo[-1])
        if busca_link(conteudo[-1])
        else busca_link(conteudo[-2])
    )
    data = transforma_data(link)
    data_limite = datetime(2022, 1, 1)
    print(data)
    if data is not None and data < data_limite:
        limit = False
        break




2024-04-16 00:00:00
2024-04-15 00:00:00
2024-04-06 00:00:00
2024-04-05 00:00:00
2024-04-04 00:00:00
2024-03-28 00:00:00
2024-03-20 00:00:00
2024-03-14 00:00:00
2024-03-12 00:00:00
2024-03-11 00:00:00
2024-03-10 00:00:00
2024-03-01 00:00:00
2024-02-09 00:00:00
2024-02-05 00:00:00
2024-01-31 00:00:00
2024-01-26 00:00:00
2024-01-21 00:00:00
2024-01-16 00:00:00
2024-01-10 00:00:00
2024-01-04 00:00:00
2023-12-27 00:00:00


In [9]:


# Diretório de trabalho atual
working_dir = Path(os.getcwd())

# Diretório de saída relativo ao diretório de trabalho atual
output_dir = working_dir / 'output'

# Verifica se o diretório existe e, se não existir, cria-o
output_dir.mkdir(parents=True, exist_ok=True)




dados = []

for i, titulo in enumerate(conteudo):
    link = busca_link(conteudo[i])
    titulo_texto = obtem_titulo(conteudo[i])
    data = transforma_data(link)

    dados.append((data, titulo_texto, link))

df = pd.DataFrame(dados, columns=["data", "texto", "Link"])



# Agora você pode salvar o arquivo Excel no diretório de saída
df.to_excel(output_dir / 'dados.xlsx', index=False)



In [10]:
df.head()

,data,texto,Link
0,2024-04-30,"Petrobras atrasa reajustes nos combustíveis, m...",https://g1.globo.com/economia/noticia/2024/04/...
1,2024-04-30,"Dólar salta 1,5% e fecha próximo de R$ 5,20, à...",https://g1.globo.com/economia/noticia/2024/04/...
2,2024-04-30,Petrobras e governo do RN assinam acordo para ...,https://g1.globo.com/rn/rio-grande-do-norte/no...
3,2024-04-26,"Ibovespa tem alta de 1,51% com melhora de Petr...",https://g1.globo.com/economia/noticia/2024/04/...
4,2024-04-26,Por que Petrobras é petroleira que mais paga d...,https://g1.globo.com/economia/negocios/noticia...
